In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {*** # имя пользователя
***, # пароль
***,
***: 6432, # порт подключения
***: 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# Введение



**Проект по SQL**

---
**Описание исследования:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель исследования.**

* Ответить на вопросы:
1) Посчитайте, сколько книг вышло после 1 января 2000 года;
2) Для каждой книги посчитайте количество обзоров и среднюю оценку;
3) Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4) Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5) Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

---

**Задачи исследования.**

- Изучить источники данных;
- Написать запросы, которые выведут на экран ответы на вопросы исследования. 

---

**Исходные данные.**  

Таблица `books` — содержит данные о книгах:  

`book_id` — идентификатор книги;
`author_id` — идентификатор автора;
`title` — название книги;
`num_pages` — количество страниц;
`publication_date` — дата публикации книги;
`publisher_id` — идентификатор издателя.   


Таблица `authors` — содержит данные об авторах:

`author_id` — идентификатор автора;
`author` — имя автора.


Таблица `publishers` — содержит данные об издательствах:

`publisher_id` — идентификатор издательства;
`publisher` — название издательства;  


Таблица `ratings` — содержит данные о пользовательских оценках книг:

`rating_id` — идентификатор оценки;
`book_id` — идентификатор книги;
`username` — имя пользователя, оставившего оценку;
`rating` — оценка книги. 

Таблица `reviews` — содержит данные о о пользовательских обзорах:

`review_id` — идентификатор обзора;
`book_id` — идентификатор книги;
`username` — имя автора обзора;
`text` — текст обзора.

---
**План работы**  

Часть 1. Изучение данных.  
Открываем файл и ознакамливаемся с данными и их структурой.   

Часть 2. Ответы на вопросы исследования.

## Изучение и предобработка данных

Ознакомимся поочерёдно с каждой таблицей:

In [3]:
books=pd.io.sql.read_sql("SELECT * FROM books", con = engine) 
books.head(10)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [4]:
authors = pd.io.sql.read_sql("SELECT * FROM authors", con = engine) 
authors.head(10)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [5]:
publishers = pd.io.sql.read_sql("SELECT * FROM publishers", con = engine) 
publishers.head(10)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [6]:
ratings = pd.io.sql.read_sql("SELECT * FROM ratings", con = engine) 
ratings.head(10)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [7]:
reviews = pd.io.sql.read_sql("SELECT * FROM reviews", con = engine) 
reviews.head(10)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


## Ответы на вопросы исследования.

### Cколько книг вышло после 1 января 2000 года?

In [8]:
pd.io.sql.read_sql('''

SELECT COUNT(title) 
FROM books
WHERE publication_date>'2000-01-01'

''', con = engine) 

,count
0,819


Начиная со второго января 2000 года вышло 819 книг.

### Количество обзовор и средняя оценка для каждой книги:

In [9]:
pd.io.sql.read_sql('''

SELECT b.book_id,
COUNT(re.review_id),
AVG(r.rating)
FROM books as b
LEFT JOIN ratings AS r on b.book_id=r.book_id
LEFT JOIN reviews AS re on b.book_id=re.book_id
GROUP BY b.book_id
ORDER BY COUNT(re.review_id) DESC
''', con = engine) 

,book_id,count,avg
0,948,1120,3.662500
1,750,528,4.125000
2,673,516,3.825581
3,302,492,4.414634
4,299,480,4.287500
...,...,...,...
995,221,0,4.000000
996,808,0,3.000000
997,191,0,4.000000
998,83,0,3.666667


При запросы были использованы левые джойны, т.к. иначе книги без обзоров бы не сохранились в результатах.

### Какое издательство выпустило больше всего книг толще 50 страниц?

In [10]:
pd.io.sql.read_sql('''

SELECT p.publisher,
COUNT(b.book_id)
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages>50
GROUP BY p.publisher
ORDER BY COUNT(book_id) DESC
LIMIT 1

''', con = engine) 

,publisher,count
0,Penguin Books,42


Больше всего книг- 42 было выпущено изданием "Penguin Books".

### Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [11]:
pd.io.sql.read_sql('''
SELECT r.username,
AVG(r.rating)
FROM (SELECT book_id,
COUNT(book_id)
FROM ratings
GROUP BY book_id
HAVING COUNT(book_id)>=50) AS a
JOIN ratings AS r ON r.book_id = a.book_id
GROUP BY r.username
ORDER BY AVG(r.rating) DESC
LIMIT 1

''', con = engine)

,username,avg
0,martinyvonne,4.8


Самая высокая средняя оценка - 4,8 балла принадлежит пользователю с ником "martinyvonne"

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
pd.io.sql.read_sql('''

SELECT AVG (reviews)
FROM(SELECT re.username,
COUNT(DISTINCT(re.review_id)) as reviews
FROM ratings AS ra
JOIN reviews AS re ON ra.username = re.username
GROUP BY re.username
HAVING COUNT(DISTINCT(ra.rating_id))>50) AS a

''', con = engine)

,avg
0,24.333333


В среднем пользователи, поставившие более 50 оценок делают 24,3 обзора.

## Вывод

В ходе исследования были изучены таблицы и найдены ответы на вопросы:  

* Начиная со второго января 2000 года вышло 819 книг.  
* Посчитано количество обзоров и средняя оценка для каждой книги.  
* Больше всего книг- 42 было выпущено изданием "Penguin Books".  
* Самая высокая средняя оценка - 4,8 балла принадлежит пользователю с ником "martinyvonne"  
* В среднем пользователи, поставившие более 50 оценок делают 24,3 обзора.  

Исследование можно считать завершённым.